In [ ]:
!pip install scikit-learn pandas numpy matplotlib seaborn

In [ ]:
from google.colab import files
uploaded = files.upload() # link to dataset https://github.com/AlihamzaMaan/Random-Forest-Classification/blob/main/dataset_1.csv

In [ ]:
import pandas as pd

df = pd.read_csv("modeling_dataset.csv")
print(df.head())  # Display the first few rows

   Unnamed: 0  packet_count  total_bytes  avg_packet_size  \
0           0          4155    6034168.0      1452.266667   
1           1          3388    4850245.0      1431.595336   
2           2          3664    5287877.0      1443.197871   
3           3          4108    5948521.0      1448.033350   
4           4          4412    6419061.0      1454.909565   

   avg_inter_packet_delay  std_inter_packet_delay  chunk_count  avg_ssim  \
0                1.657439                5.760497           14  0.972431   
1                2.218187                9.324953           10  0.971165   
2                1.985804                7.874238           10  0.971165   
3                1.688093                5.893905           11  0.972424   
4                1.531172                4.442955           11  0.980529   

     avg_format  avg_chunk_size  
0  1920x1080-22   401506.214286  
1  1920x1080-22   499983.900000  
2  1920x1080-22   499983.900000  
3  1920x1080-22   509411.000000  
4  192

In [ ]:
print(df.info())  # View dataset structure
print(df.describe())  # Summary statistics

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2176 entries, 0 to 2175
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Unnamed: 0              2176 non-null   int64  
 1   packet_count            2176 non-null   int64  
 2   total_bytes             2176 non-null   float64
 3   avg_packet_size         2176 non-null   float64
 4   avg_inter_packet_delay  2176 non-null   float64
 5   std_inter_packet_delay  2176 non-null   float64
 6   chunk_count             2176 non-null   int64  
 7   avg_ssim                2176 non-null   float64
 8   avg_format              2176 non-null   object 
 9   avg_chunk_size          2176 non-null   float64
dtypes: float64(6), int64(3), object(1)
memory usage: 170.1+ KB
None
        Unnamed: 0  packet_count   total_bytes  avg_packet_size  \
count  2176.000000   2176.000000  2.176000e+03      2176.000000   
mean   1087.500000   2622.931985  3.768944e+06      14

In [ ]:
# Define features (X) and target (y)
X = df.drop(columns=["avg_format", "avg_ssim"])  # target is the name of the column we are targeting, we can change this depending on which metric we are tsting for
y = df["avg_ssim"]

# Split data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score


# Initialize and train the model
# model = RandomForestClassifier(n_estimators=100,
#                                random_state=42,
#                                min_samples_leaf=5,
#                                max_depth=10 )
model = RandomForestRegressor(n_estimators=50,
                              random_state=42,
                              min_samples_leaf=5,
                              max_depth=20)
model.fit(X_train, y_train)


RandomForestRegressor(max_depth=20, min_samples_leaf=5, n_estimators=50,
                      random_state=42)

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
# Predict on the test set
y_pred = model.predict(X_test)
# Check if you're accidentally using training data
print("MSE on training set:", mean_squared_error(y_train, model.predict(X_train)))
print("MSE on test set:", mean_squared_error(y_test, model.predict(X_test)))

mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse:.2f}")
# Calculate accuracyf
#accuracy = accuracy_score(y_test, y_pred)
#print(f"Model Accuracy: {accuracy:.2f}")

MSE on training set: 1.5567204354316182e-05
MSE on test set: 2.3092637834930362e-05
Mean Squared Error: 0.00


In [19]:
!pip install trustee

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from trustee import RegressionTrustee

#Load dataset
df = pd.read_csv("/content/modeling_dataset.csv")
#Drop rows with missing target
df = df.dropna(subset=['avg_ssim'])
#Define features and target
features = [
    'packet_count',
    'total_bytes',
    'avg_packet_size',
    'avg_inter_packet_delay',
    'std_inter_packet_delay'
]
X = df[features].fillna(0)
y = df['avg_ssim']  #continuous values for regression
#Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
#Train Random Forest Regressor
reg = RandomForestRegressor(n_estimators=100, random_state=42)
reg.fit(X_train, y_train)
#Predict and evaluate
y_pred = reg.predict(X_test)
print("MSE:", mean_squared_error(y_test, y_pred))
print("R² Score:", r2_score(y_test, y_pred))
#Explain model using RegressionTrustee
trustee = RegressionTrustee(expert=reg)
trustee.fit(X_train, y_train, num_iter=50, num_stability_iter=10, samples_size=0.3, verbose=True)
dt, pruned_dt, agreement, reward = trustee.explain()
dt_y_pred = dt.predict(X_test)
#Evaluate fidelity (tree vs expert) and performance (tree vs ground truth)
print("\n Fidelity (Tree vs Forest):")
print("MSE:", mean_squared_error(y_pred, dt_y_pred))
print("R²:", r2_score(y_pred, dt_y_pred))

print("\n Performance (Tree vs Ground Truth):")
print("MSE:", mean_squared_error(y_test, dt_y_pred))
print("R²:", r2_score(y_test, dt_y_pred))

MSE: 0.00020106395838638221
R² Score: 0.28617213360266547
Initializing training dataset using RandomForestRegressor(random_state=42) as expert model
Expert model score: 0.899494199568325
Initializing Trustee outer-loop with 10 iterations
########## Outer-loop Iteration 0/10 ##########
Initializing Trustee inner-loop with 10 iterations
########## Inner-loop Iteration 0/50 ##########
Sampling 365 points from training dataset with (1218, 1218) entries
Student model 0-0 trained with depth 22 and 255 leaves:
Student model score: 0.21644178154854266
Student model 0-0 fidelity: 0.21644178154854266
########## Inner-loop Iteration 1/50 ##########
Sampling 365 points from training dataset with (1328, 1328) entries
Student model 0-1 trained with depth 20 and 251 leaves:
Student model score: 0.30523238611428816
Student model 0-1 fidelity: 0.30523238611428816
########## Inner-loop Iteration 2/50 ##########
Sampling 365 points from training dataset with (1438, 1438) entries
Student model 0-2 trained

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but DecisionTreeRegressor was fitted without feature names
  warnings.warn(


In [21]:
import os
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from trustee.report.trust import TrustReport
from sklearn.utils.multiclass import check_classification_targets

# === Load your dataset ===
df = pd.read_csv("/content/modeling_dataset.csv")
df = df.dropna(subset=["avg_ssim"])  # drop missing regression target

# === Define features and target ===
features = [
    'packet_count',
    'total_bytes',
    'avg_packet_size',
    'avg_inter_packet_delay',
    'std_inter_packet_delay'
]
X = df[features].fillna(0)
y = df["avg_ssim"]

# === Manually split the data ===
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)

# === Fit the blackbox regressor ===
reg = RandomForestRegressor(n_estimators=100, random_state=42)
reg.fit(X_train, y_train)

# === Run TrustReport ===
trust_report = TrustReport(
    reg,
    X=X,
    y=y,
    X_train=X_train,
    y_train=y_train,
    X_test=X_test,
    y_test=y_test,
    max_iter=5,
    num_pruning_iter=0,
    trustee_num_iter=10,
    trustee_num_stability_iter=5,
    trustee_sample_size=0.3,
    analyze_branches=True,
    analyze_stability=True,
    top_k=10,
    verbose=True,
    feature_names=features,
    is_classify=False,  # regression mode
)

# === Save report ===
OUTPUT_PATH = "out/"
REPORT_PATH = f"{OUTPUT_PATH}/report/trust_report.obj"
os.makedirs(os.path.dirname(REPORT_PATH), exist_ok=True)
trust_report.save(REPORT_PATH)

print(" Trust report generated and saved.")
print(trust_report)


Running Trust Report...
Preparing data...
Done!
Progress |----------------------------------------------------------------------------------------------------| 0.9% Complete
Done!
Progress |█---------------------------------------------------------------------------------------------------| 1.8% Complete
Fitting blackbox model...
Done!
Blackbox model score report with training data:
R2 Score: 0.26841767821796003
Using Classification Trustee algorithm to extract DT...
Initializing training dataset using RandomForestRegressor(random_state=42) as expert model
Expert model score: 0.8998362204506755
Initializing Trustee outer-loop with 5 iterations
########## Outer-loop Iteration 0/5 ##########
Initializing Trustee inner-loop with 5 iterations
########## Inner-loop Iteration 0/10 ##########
Sampling 319 points from training dataset with (1066, 1066) entries
Student model 0-0 trained with depth 22 and 223 leaves:
Student model score: 0.3334193301915257
Student model 0-0 fidelity: 0.333419330

Done!
Plotting...


Streaming output truncated to the last 5000 lines.


Done!
Done!
✅ Trust report generated and saved.

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                   Classification Trust Report                                                                                                                    |
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                           +-----------------------------------------------------------------------------------------------------------

<Figure size 3000x300 with 0 Axes>

...

<Figure size 5000x300 with 0 Axes>

<Figure size 5000x300 with 0 Axes>

<Figure size 4000x300 with 0 Axes>

<Figure size 4000x300 with 0 Axes>

<Figure size 4000x300 with 0 Axes>